# ⭐️ 목표 : 지하철 역을 기준으로 월별 평균 승하차합계 및 지하철 위치에 대한 정보를 담고있는 데이터 생성
- 날짜, 지하철역별, 시간대별 승하차 인원 통계 데이터와 지하철 위치 데이터를 병합하기
- 월평균승하차합계가 높은 역을 확인하기

In [5]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from glob import glob
from tqdm import tqdm
from haversine import haversine
import geopy.distance
import numpy as np
import random
import folium
from folium.features import CustomIcon
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import platform
import requests
import json
try:

    if platform.system() == 'Windows':

        #윈도우인 경우

        font_name = font_manager.FontProperties(fname="c:/Windows/fonts/malgun.ttf").get_name()

        rc('font', family=font_name)

    else:

        #Mac인 경우

        rc('font', family='AppleGothic')

except:

    pass

matplotlib.rcParams['axes.unicode_minus'] = False

## ✅ 데이터 전처리

In [6]:
station = pd.read_csv('/Users/nabong/Desktop/project_login/data/subway/staion_population_daily20221231.csv', encoding='cp949')
station_loc = pd.read_csv('/Users/nabong/Desktop/project_login/data/subway/seoul_subway_location.csv', encoding='cp949')

/var/folders/5z/x3njjl9j3tb_1llqpqvxxqhm0000gn/T/ipykernel_61244/2668084604.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  station = pd.read_csv('/Users/nabong/Desktop/project_login/data/subway/staion_population_daily20221231.csv', encoding='cp949')


In [7]:
# 역별 승하차 데이터 구조 확인
station.info()
station.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199080 entries, 0 to 199079
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   연번            199080 non-null  int64  
 1   수송일자          199080 non-null  object 
 2   호선            199080 non-null  int64  
 3   고유역번호(외부역코드)  199080 non-null  object 
 4   역명            199080 non-null  object 
 5   승하차구분         199080 non-null  object 
 6   06시이전         199080 non-null  int64  
 7   06-07시간대      199080 non-null  int64  
 8   07-08시간대      199080 non-null  int64  
 9   08-09시간대      199080 non-null  int64  
 10  09-10시간대      199080 non-null  int64  
 11  10-11시간대      199080 non-null  int64  
 12  11-12시간대      199080 non-null  int64  
 13  12-13시간대      199080 non-null  int64  
 14  13-14시간대      199080 non-null  int64  
 15  14-15시간대      199080 non-null  int64  
 16  15-16시간대      199080 non-null  int64  
 17  16-17시간대      199080 non-null  int64  
 18  17-1

,연번,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
0,1,2022-01-01,1,150,서울역,승차,120,137,211,439,...,1566,1686,1591,1358,1062,899,1327,814,234,NaN
1,2,2022-01-01,1,150,서울역,하차,113,560,617,910,...,1329,1251,1126,884,764,654,728,416,131,NaN
2,3,2022-01-01,1,151,시청,승차,38,66,101,139,...,474,550,672,528,420,434,491,232,38,NaN
3,4,2022-01-01,1,151,시청,하차,31,195,224,380,...,408,377,354,213,131,98,137,61,24,NaN
4,5,2022-01-01,1,152,종각,승차,44,71,86,158,...,889,964,1024,803,855,1099,1209,255,62,NaN


In [8]:
# 역 위치 데이터 구조 확인
station_loc.info()
station_loc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   765 non-null    int64  
 1   역사명     765 non-null    object 
 2   호선      765 non-null    object 
 3   위도      765 non-null    float64
 4   경도      765 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.0+ KB


,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387


In [4]:
# 데이터 타입이 object인 컬럼 중 고유역번호 데이터 확인 - 확인결과 int타입과 string 타입이 섞여있고 공백문자가 들어간 컬럼도 있음
print(station['고유역번호(외부역코드)'].unique())
print(station['고유역번호(외부역코드)'].unique().shape[0])

[150 151 152 153 154 155 156 157 158 159 201 202 203 204 205 206 207 208
 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226
 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244
 245 246 247 248 249 250 309 310 311 312 313 314 315 316 317 318 319 320
 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339
 340 341 342 409 410 411 412 413 414 415 416 417 418 419 420 421 422 423
 424 425 426 427 428 429 430 431 432 433 434 2511 2512 2513 2514 2515 2516
 2517 2518 2519 2520 2521 2522 2523 2524 2525 2526 2527 2528 2529 2530
 2531 2532 2533 2534 2535 2536 2537 2538 2539 2540 2541 2542 2543 2544
 2545 2546 2547 2548 2549 2550 2551 2552 2553 2554 2555 2556 2557 2558
 2559 2560 2561 2562 2563 2564 2565 2566 2611 2612 2613 2614 2616 2617
 2618 2619 2620 2621 2622 2623 2624 2625 2626 2627 2628 2629 2630 2631
 2632 2633 2634 2635 2636 2637 2638 2639 2640 2641 2642 2643 2644 2645
 2646 2647 2648 2711 2712 2713 2714 2715 2716 2717 2718 2719 

In [5]:
# 고유역코드가 비어있는 컬럼은 1개이고 광명사거리역
station[station['고유역번호(외부역코드)'] == ' ']

,연번,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
132480,132481,2022-08-31,7,,광명사거리,승차,801,1299,3405,3901,...,965,1181,1376,1283,779,521,448,346,124,15.0


In [6]:
# 광명사거리의 고유역번호를 확인
station[station['역명'] == '광명사거리'].head(2)

,연번,수송일자,호선,고유역번호(외부역코드),역명,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
502,503,2022-01-01,7,2750,광명사거리,승차,208,208,315,417,...,852,866,710,589,359,408,415,176,58,NaN
503,504,2022-01-01,7,2750,광명사거리,하차,50,122,179,258,...,822,771,771,742,509,456,678,556,235,NaN


In [7]:
# 같은 7호선이며 고유역코드가 2750임이 확인되어 2750으로 대치
station['고유역번호(외부역코드)'].replace(' ', '2750', inplace=True)
station[station['고유역번호(외부역코드)'] == ' '].shape[0] # 대치가 완료되어 해당 데이터 없음

0

In [8]:
# 공통 컬럼인 고유역번호(외부역코드)의 데이터 타입이 다르므로 object를 통일하고 이름도 node_id 변경(버스정보와 합치기위해 통일)
# 수송일자 컬럼의 데이터타입을 날짜로 변경하고 호선의 데이터타입을 string으로 변경
station.rename(columns={'고유역번호(외부역코드)':'node_id', '역명':'정류소명(역명)'}, inplace=True)
station_loc.rename(columns={'역사_ID':'node_id', '역사명':'정류소명(역명)'}, inplace=True)


station = station.astype({'node_id':'string', '수송일자':'datetime64', '호선':'string', '정류소명(역명)':'string'})
station_loc = station_loc.astype({'node_id':'string', '정류소명(역명)':'string', '호선':'string'})

# 바뀐 내용 확인
station.info()
station_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199080 entries, 0 to 199079
Data columns (total 26 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   연번        199080 non-null  int64         
 1   수송일자      199080 non-null  datetime64[ns]
 2   호선        199080 non-null  string        
 3   node_id   199080 non-null  string        
 4   정류소명(역명)  199080 non-null  string        
 5   승하차구분     199080 non-null  object        
 6   06시이전     199080 non-null  int64         
 7   06-07시간대  199080 non-null  int64         
 8   07-08시간대  199080 non-null  int64         
 9   08-09시간대  199080 non-null  int64         
 10  09-10시간대  199080 non-null  int64         
 11  10-11시간대  199080 non-null  int64         
 12  11-12시간대  199080 non-null  int64         
 13  12-13시간대  199080 non-null  int64         
 14  13-14시간대  199080 non-null  int64         
 15  14-15시간대  199080 non-null  int64         
 16  15-16시간대  199080 non-null  int64      

In [9]:
# 서울 외 지역의 지하철역 확인을 위해 주소 불러오기
# api 키 불러오기
with open('/Users/nabong/Desktop/api_keys/rest_api.txt', 'r') as f:
    api_key = f.read()

# 좌표를 기반으로 주소 불러오기
gps = list(zip(station_loc['경도'], station_loc['위도']))
headers = {'Authorization':'KakaoAK '+api_key}

addr = []
for g in tqdm(gps):
    x_val = g[0]
    y_val = g[1]
    base_url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x='+str(x_val)+'&y='+str(y_val)
    api_req = requests.get(base_url, headers=headers)
    addr.append(json.loads(api_req.text))

len(addr)

100%|█████████████████████████████████████████| 765/765 [03:40<00:00,  3.48it/s]


765

In [10]:
addr

[{'meta': {'total_count': 2},
  'documents': [{'region_type': 'B',
    'code': '4145010900',
    'address_name': '경기도 하남시 망월동',
    'region_1depth_name': '경기도',
    'region_2depth_name': '하남시',
    'region_3depth_name': '망월동',
    'region_4depth_name': '',
    'x': 127.1894750978691,
    'y': 37.56809946143883},
   {'region_type': 'H',
    'code': '4145061000',
    'address_name': '경기도 하남시 미사1동',
    'region_1depth_name': '경기도',
    'region_2depth_name': '하남시',
    'region_3depth_name': '미사1동',
    'region_4depth_name': '',
    'x': 127.18880799273204,
    'y': 37.56031593309407}]},
 {'meta': {'total_count': 2},
  'documents': [{'region_type': 'B',
    'code': '1174011000',
    'address_name': '서울특별시 강동구 강일동',
    'region_1depth_name': '서울특별시',
    'region_2depth_name': '강동구',
    'region_3depth_name': '강일동',
    'region_4depth_name': '',
    'x': 127.17391966164308,
    'y': 37.564944692060216},
   {'region_type': 'H',
    'code': '1174052600',
    'address_name': '서울특별시 강동구 상일2동',
  

In [11]:
# 받아 온 주소에서 시를 추출하기
si_list = []
for i in range(len(addr)):
    si = addr[i]['documents'][0]['region_1depth_name']
    si_list.append(si)

print(len(si_list))
print(si_list[:4])

765
['경기도', '서울특별시', '서울특별시', '경기도']


In [12]:
set(si_list)

{'강원도', '경기도', '서울특별시', '인천광역시', '충청남도'}

In [13]:
# 추출한 시를 데이터프레임에 '지역'으로 추가하기
station_loc['지역'] = si_list
station_loc.head()

,node_id,정류소명(역명),호선,위도,경도,지역
0,9996,미사,5호선,37.560927,127.193877,경기도
1,9995,강일,5호선,37.557490,127.175930,서울특별시
2,4929,김포공항,김포골드라인,37.562360,126.801868,서울특별시
3,4928,고촌,김포골드라인,37.601243,126.770345,경기도
4,4927,풍무,김포골드라인,37.612488,126.732387,경기도


In [14]:
# 전체 데이터의 지하철 역의 수 : 765개
# 서울 외 지역의 지하철 역의 수 : 364개
out_of_seoul = station_loc[station_loc['지역'] != '서울특별시']
out_of_seoul

,node_id,정류소명(역명),호선,위도,경도,지역
0,9996,미사,5호선,37.560927,127.193877,경기도
3,4928,고촌,김포골드라인,37.601243,126.770345,경기도
4,4927,풍무,김포골드라인,37.612488,126.732387,경기도
5,4926,사우(김포시청),김포골드라인,37.620249,126.719731,경기도
6,4925,걸포북변,김포골드라인,37.631650,126.705975,경기도
...,...,...,...,...,...,...
606,1205,구리,중앙선,37.603392,127.143869,경기도
668,408,별내별가람,진접선,37.667780,127.115810,경기도
669,406,오남,진접선,37.705000,127.192810,경기도
670,405,진접,진접선,37.720500,127.203400,경기도


In [15]:
# 서울 외 지역의 지하철 역 드랍 : 765 - 364 = 401
drop_index = out_of_seoul.index
station_loc = station_loc.drop(drop_index, axis=0)
station_loc

,node_id,정류소명(역명),호선,위도,경도,지역
1,9995,강일,5호선,37.557490,127.175930,서울특별시
2,4929,김포공항,김포골드라인,37.562360,126.801868,서울특별시
24,4713,신설동,우이신설선,37.576095,127.023242,서울특별시
25,4712,보문,우이신설선,37.585286,127.019381,서울특별시
26,4711,성신여대입구(돈암),우이신설선,37.592467,127.016516,서울특별시
...,...,...,...,...,...,...
760,154,종로5가,1호선,37.570926,127.001849,서울특별시
761,153,종로3가,1호선,37.570406,126.991847,서울특별시
762,152,종각,1호선,37.570161,126.982923,서울특별시
763,151,시청,1호선,37.565715,126.977088,서울특별시


In [16]:
# 데이터 드롭이 잘 됐는지 확인
station_loc['지역'].value_counts()
# 지역 컬럼 드롭
station_loc = station_loc.drop('지역', axis=1)
station_loc

,node_id,정류소명(역명),호선,위도,경도
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
24,4713,신설동,우이신설선,37.576095,127.023242
25,4712,보문,우이신설선,37.585286,127.019381
26,4711,성신여대입구(돈암),우이신설선,37.592467,127.016516
...,...,...,...,...,...
760,154,종로5가,1호선,37.570926,127.001849
761,153,종로3가,1호선,37.570406,126.991847
762,152,종각,1호선,37.570161,126.982923
763,151,시청,1호선,37.565715,126.977088


In [17]:
# 결측치를 갖고있는 24시이후 컬럼의 경우 결측치를 0으로 채워줌
station['24시이후'].fillna(0, inplace=True)
station['24시이후'] = station['24시이후'].astype('int')

In [18]:
# 승하차 데이터 나누기
station_in = station[station['승하차구분'] == '승차'].drop('승하차구분', axis=1)
station_out = station[station['승하차구분'] == '하차'].drop('승하차구분', axis=1)

In [19]:
station_in.head(2)

,연번,수송일자,호선,node_id,정류소명(역명),06시이전,06-07시간대,07-08시간대,08-09시간대,09-10시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
0,1,2022-01-01,1,150,서울역,120,137,211,439,592,...,1566,1686,1591,1358,1062,899,1327,814,234,0
2,3,2022-01-01,1,151,시청,38,66,101,139,121,...,474,550,672,528,420,434,491,232,38,0


In [20]:
# 승하차 테이블에 일일 합계를 위한 '합계' 컬럼 추가
station_in['합계'] = station_in.iloc[:, 4:].sum(axis=1)
station_out['합계'] = station_out.iloc[:, 4:].sum(axis=1)

/var/folders/5z/x3njjl9j3tb_1llqpqvxxqhm0000gn/T/ipykernel_34707/3194259065.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  station_in['합계'] = station_in.iloc[:, 4:].sum(axis=1)
/var/folders/5z/x3njjl9j3tb_1llqpqvxxqhm0000gn/T/ipykernel_34707/3194259065.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  station_out['합계'] = station_out.iloc[:, 4:].sum(axis=1)


In [21]:
station_out.head(2)

,연번,수송일자,호선,node_id,정류소명(역명),06시이전,06-07시간대,07-08시간대,08-09시간대,09-10시간대,...,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후,합계
1,2,2022-01-01,1,150,서울역,113,560,617,910,1232,...,1251,1126,884,764,654,728,416,131,0,17087
3,4,2022-01-01,1,151,시청,31,195,224,380,283,...,377,354,213,131,98,137,61,24,0,5063


In [22]:
# 승하차 1년 총 합계
sum_in = pd.DataFrame(station_in.groupby(['node_id', '정류소명(역명)'])['합계'].sum()).reset_index()
sum_out = pd.DataFrame(station_out.groupby(['node_id', '정류소명(역명)'])['합계'].sum()).reset_index()

# 합계를 12로 나누고 월평균승차합계로 컬럼명을 바꾸기
sum_in['월평균승차수'] = round(sum_in['합계']/12, 2)
sum_in.drop('합계', axis=1, inplace=True)
sum_out['월평균하차수'] = round(sum_out['합계']/12, 2)
sum_out.drop('합계', axis=1, inplace=True)

# 승하차 정보를 모두 담은 하나의 테이블로 병합
sum_total = pd.merge(sum_in, sum_out, on=['node_id', '정류소명(역명)'])
# 월평균승하차총합계 컬럼생성
sum_total['월평균승하차총계'] = sum_total['월평균승차수'] + sum_total['월평균하차수']
sum_total

,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계
0,150,서울역,1301736.50,1264329.75,2566066.25
1,151,시청,625330.50,632575.33,1257905.83
2,152,종각,945011.25,914613.92,1859625.17
3,153,종로3가,727878.00,665596.33,1393474.33
4,154,종로5가,628529.42,624630.50,1253159.92
...,...,...,...,...,...
274,430,이촌(국립중앙박물관),246369.92,270050.33,516420.25
275,431,동작(현충원),55985.33,65523.33,121508.66
276,432,총신대입구(이수),520579.33,566208.00,1086787.33
277,433,사당,730378.25,628545.25,1358923.50


In [30]:
# 승하차 집계 테이블 + 위치 테이블 병합
result = pd.merge(sum_total, station_loc[['node_id', '정류소명(역명)', '위도', '경도']], on='node_id', how='outer')
result.info()
result

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 0 to 419
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   node_id     420 non-null    string 
 1   정류소명(역명)_x  279 non-null    string 
 2   월평균승차수      279 non-null    float64
 3   월평균하차수      279 non-null    float64
 4   월평균승하차총계    279 non-null    float64
 5   정류소명(역명)_y  401 non-null    string 
 6   위도          401 non-null    float64
 7   경도          401 non-null    float64
dtypes: float64(5), string(3)
memory usage: 29.5 KB


,node_id,정류소명(역명)_x,월평균승차수,월평균하차수,월평균승하차총계,정류소명(역명)_y,위도,경도
0,150,서울역,1301736.50,1264329.75,2566066.25,서울역,37.556228,126.972135
1,151,시청,625330.50,632575.33,1257905.83,시청,37.565715,126.977088
2,152,종각,945011.25,914613.92,1859625.17,종각,37.570161,126.982923
3,153,종로3가,727878.00,665596.33,1393474.33,종로3가,37.570406,126.991847
4,154,종로5가,628529.42,624630.50,1253159.92,종로5가,37.570926,127.001849
...,...,...,...,...,...,...,...,...
415,1005,<NA>,NaN,NaN,NaN,대방,37.513342,126.926382
416,1004,<NA>,NaN,NaN,NaN,노량진,37.514149,126.942710
417,1003,<NA>,NaN,NaN,NaN,용산,37.529849,126.964561
418,1002,<NA>,NaN,NaN,NaN,남영,37.541021,126.971300


In [31]:
# 역명이 없는 경우 위치데이터의 역명을 사용하면 모두 채울 수 있으므로 정류소명(역명)_y를 정류소명(역명)로 바꾸고 정류소명(역명)_x는 드랍
result['정류소명(역명)_x'] = result['정류소명(역명)_y']
result = result.rename(columns={'정류소명(역명)_x':'정류소명(역명)'}).drop('정류소명(역명)_y', axis=1)
result

,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계,위도,경도
0,150,서울역,1301736.50,1264329.75,2566066.25,37.556228,126.972135
1,151,시청,625330.50,632575.33,1257905.83,37.565715,126.977088
2,152,종각,945011.25,914613.92,1859625.17,37.570161,126.982923
3,153,종로3가,727878.00,665596.33,1393474.33,37.570406,126.991847
4,154,종로5가,628529.42,624630.50,1253159.92,37.570926,127.001849
...,...,...,...,...,...,...,...
415,1005,대방,NaN,NaN,NaN,37.513342,126.926382
416,1004,노량진,NaN,NaN,NaN,37.514149,126.942710
417,1003,용산,NaN,NaN,NaN,37.529849,126.964561
418,1002,남영,NaN,NaN,NaN,37.541021,126.971300


In [32]:
# 승하차 집계가 없는 데이터들은 드롭하지 않고 이웃역 수를 구하기 위해 내버려둠
# 위치 정보가 없는 역들은 모두 경기도권 역이기 때문에 드랍
drop_index = result[result['위도'].isna()].index
result = result.drop(drop_index, axis=0).reset_index(drop=True)
result.info()
result

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   node_id   401 non-null    string 
 1   정류소명(역명)  401 non-null    string 
 2   월평균승차수    260 non-null    float64
 3   월평균하차수    260 non-null    float64
 4   월평균승하차총계  260 non-null    float64
 5   위도        401 non-null    float64
 6   경도        401 non-null    float64
dtypes: float64(5), string(2)
memory usage: 22.1 KB


,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계,위도,경도
0,150,서울역,1301736.50,1264329.75,2566066.25,37.556228,126.972135
1,151,시청,625330.50,632575.33,1257905.83,37.565715,126.977088
2,152,종각,945011.25,914613.92,1859625.17,37.570161,126.982923
3,153,종로3가,727878.00,665596.33,1393474.33,37.570406,126.991847
4,154,종로5가,628529.42,624630.50,1253159.92,37.570926,127.001849
...,...,...,...,...,...,...,...
396,1005,대방,NaN,NaN,NaN,37.513342,126.926382
397,1004,노량진,NaN,NaN,NaN,37.514149,126.942710
398,1003,용산,NaN,NaN,NaN,37.529849,126.964561
399,1002,남영,NaN,NaN,NaN,37.541021,126.971300


In [33]:
# 버스데이터와 병합하기 위해 구분자로 S를 추가
result['node_id'] = 'S' + result['node_id']
result.info()
result

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   node_id   401 non-null    string 
 1   정류소명(역명)  401 non-null    string 
 2   월평균승차수    260 non-null    float64
 3   월평균하차수    260 non-null    float64
 4   월평균승하차총계  260 non-null    float64
 5   위도        401 non-null    float64
 6   경도        401 non-null    float64
dtypes: float64(5), string(2)
memory usage: 22.1 KB


,node_id,정류소명(역명),월평균승차수,월평균하차수,월평균승하차총계,위도,경도
0,S150,서울역,1301736.50,1264329.75,2566066.25,37.556228,126.972135
1,S151,시청,625330.50,632575.33,1257905.83,37.565715,126.977088
2,S152,종각,945011.25,914613.92,1859625.17,37.570161,126.982923
3,S153,종로3가,727878.00,665596.33,1393474.33,37.570406,126.991847
4,S154,종로5가,628529.42,624630.50,1253159.92,37.570926,127.001849
...,...,...,...,...,...,...,...
396,S1005,대방,NaN,NaN,NaN,37.513342,126.926382
397,S1004,노량진,NaN,NaN,NaN,37.514149,126.942710
398,S1003,용산,NaN,NaN,NaN,37.529849,126.964561
399,S1002,남영,NaN,NaN,NaN,37.541021,126.971300


In [34]:
# 데이터 전처리 및 병합한 파일들을 저장
result.to_excel('subway_info.xlsx', index=False)
station.to_excel('station_pop.xlsx', index=False)
station_loc.to_excel('station_loc.xlsx', index=False)

## ✅ 시각화

In [ ]:
# 승차 인원 수가 많은 역 TOP10
sns.set_style('white')
sns.color_palette("rocket", as_cmap=True)
in_top10 = result.sort_values(by='월평균승차수', ascending=False).head(10)
sns.barplot(x='월평균승차수', y='정류소명(역명)', data=in_top10)
plt.show()

# 하차 인원 수가 많은 역 TOP10
out_top10 = result.sort_values(by='월평균하차수', ascending=False).head(10)
sns.barplot(x='월평균하차수', y='정류소명(역명)', data=out_top10)
plt.show()

# 승하차 총계가 많은 역 TOP10
total_top10 = result.sort_values(by='월평균승하차총계', ascending=False).head(10)
sns.barplot(x='월평균승하차총계', y='정류소명(역명)', data=total_top10)
plt.show()